In [2]:
!pip install gym scapy

from scapy.all import rdpcap
import pandas as pd           
import numpy as np             
from datetime import datetime
import gym

In [71]:
class SpeedTestConnection:
    def __init__(self, pcap_file):
        self.pcap_file = pcap_file
        self.connection_df = None
    def get_dataframe(self):
            """
            Assumes TCP packets only
            """
            packets = rdpcap(self.pcap_file)
            # Extract fields from each packet
            fields_list = []
            for pkt in packets:
                if pkt.haslayer('UDP') or not pkt.haslayer('IP'):
                        continue
                if pkt.haslayer('TCP') and len(pkt['TCP'].payload) > 0:
                    ip = pkt['IP']
                    tcp = pkt['TCP']
                    fields_list.append([pkt.time, ip.src, ip.dst, ip.len, ip.proto, 'TCP', tcp.sport, tcp.dport, len(tcp.payload), tcp.chksum])
            
            # Create DataFrame
            df = pd.DataFrame(fields_list, columns=['Timestamp', 'Source IP', 'Destination IP', 'Length', 'Protocol', 'Transport Protocol', 'Source Port', 'Destination Port', 'Payload Length', 'Checksum'])
    
            def epoch_to_datetime_string(epoch_time):
                dt_obj = datetime.fromtimestamp(int(epoch_time))            
                datetime_str = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
                return datetime_str
            df['Time'] = df['Timestamp'].apply(epoch_to_datetime_string)

            df = df.sort_values(by='Time')
            # csv_file_path = "cached_data/" + self.pcap_file
            # df.to_csv(csv_file_path, index=False)

            return df

In [72]:
def readPCAP(directory):    
    def list_pcap_files(folder):
        pcap_files = []
        for root, dirs, files in os.walk(folder):
            for file in files:
                if file.endswith(".pcap.gz"):
                    pcap_files.append(os.path.join(root, file))
        return pcap_files
    
    pcap_list = list_pcap_files(directory)
    connection_list = []
    for pcap in pcap_list:
        connection_list.append(SpeedTestConnection(pcap))
    return connection_list


In [73]:
connection_list = readPCAP("../sample_data/20240301T000205.745148Z-pcap-mlab2-gru02-ndt/")

In [74]:
df_conn = connection_list[2].get_dataframe()

In [75]:
connection_list[2].pcap_file

'../sample_data/20240301T000205.745148Z-pcap-mlab2-gru02-ndt/2024/03/01/ndt-bkv82_1705051090_00000000005101D2.pcap.gz'

In [76]:
df_conn

,Timestamp,Source IP,Destination IP,Length,Protocol,Transport Protocol,Source Port,Destination Port,Payload Length,Checksum,Time
0,1709251284.631233000,187.109.205.54,177.136.80.216,668,6,TCP,24703,443,28,35509,2024-03-01 05:31:24
164,1709251284.921476000,177.136.80.216,187.109.205.54,2036,6,TCP,443,24703,28,37611,2024-03-01 05:31:24
165,1709251284.923846000,177.136.80.216,187.109.205.54,8452,6,TCP,443,24703,28,44027,2024-03-01 05:31:24
166,1709251284.923851000,177.136.80.216,187.109.205.54,8452,6,TCP,443,24703,28,44027,2024-03-01 05:31:24
167,1709251284.923874000,177.136.80.216,187.109.205.54,8452,6,TCP,443,24703,28,44027,2024-03-01 05:31:24
...,...,...,...,...,...,...,...,...,...,...,...
15251,1709251295.196615000,177.136.80.216,187.109.205.54,1452,6,TCP,443,24703,28,37027,2024-03-01 05:31:35
15252,1709251295.196619000,177.136.80.216,187.109.205.54,8452,6,TCP,443,24703,28,44027,2024-03-01 05:31:35
15253,1709251295.197412000,177.136.80.216,187.109.205.54,4252,6,TCP,443,24703,28,39827,2024-03-01 05:31:35
15255,1709251295.201453000,177.136.80.216,187.109.205.54,1452,6,TCP,443,24703,28,37027,2024-03-01 05:31:35


In [ ]:
# dataset OK 
# rl 